# My Spotify Data
A project to analyze my own spotify data and see what I can learn from it.

List of things I want to do:
1. create dataframe
2. do monthly playlists show pattterns?
3. what charactersistics have my favourite songs/artists/...

4. update function
    - update dataframe with new playlists from my library
    - only add new data


overall: 
- create exploratory analysis and get recommended playlist for a month based on my listening habits
- create dashboard/overview/... that displays all the interesting stuff in a visually appealing manner


In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

import os
from dotenv import load_dotenv

import math
import pandas as pd
import numpy as np


## 1 Spotify Data
using ```spotipy``` library to get my spotify data.

## 1.1 authentication

In [ ]:
# scope = "user-library-read"	
scope = "playlist-read-private"

load_dotenv()  
id = os.getenv("SPTFY_ID")
secret = os.getenv("SPTFY_SECRET")

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=id,
                                               client_secret=secret,
                                               redirect_uri="http://localhost:8080/",
                                               scope=scope))



c9a30106e12d4683aeec87d37a2749e6


## 1.2 get playlists

create dictionary, containing the name and id of each playlist

In [3]:
results = sp.current_user_playlists(limit=50)

my_playlists = {}

for i in range(math.ceil(results['total']/50)):
    playlist_results = sp.current_user_playlists(limit=50, offset=i*50)
    for playlist in playlist_results['items']:
        my_playlists[playlist['name']] = playlist['id']


In [4]:
results

{'href': 'https://api.spotify.com/v1/users/*lotti_karotti*/playlists?offset=0&limit=50',
 'limit': 50,
 'next': 'https://api.spotify.com/v1/users/*lotti_karotti*/playlists?offset=50&limit=50',
 'offset': 0,
 'previous': None,
 'total': 119,
 'items': [{'collaborative': False,
   'description': '',
   'external_urls': {'spotify': 'https://open.spotify.com/playlist/6ZYkzdEakzOe8quw3tiPod'},
   'href': 'https://api.spotify.com/v1/playlists/6ZYkzdEakzOe8quw3tiPod',
   'id': '6ZYkzdEakzOe8quw3tiPod',
   'images': [{'height': 640,
     'url': 'https://mosaic.scdn.co/640/ab67616d00001e02387becb09463a8ee1aa6adacab67616d00001e0267dd241b27aee590aea08a66ab67616d00001e028ecc33f195df6aa257c39eaaab67616d00001e029aef83404b80b93c45a03b11',
     'width': 640},
    {'height': 300,
     'url': 'https://mosaic.scdn.co/300/ab67616d00001e02387becb09463a8ee1aa6adacab67616d00001e0267dd241b27aee590aea08a66ab67616d00001e028ecc33f195df6aa257c39eaaab67616d00001e029aef83404b80b93c45a03b11',
     'width': 300},
   

In [5]:
len(my_playlists)

# for i, (k, v) in enumerate(my_playlists.items()):
#     print(k, v)
#     if i > 3:
#         break

117

## 1.3 get tracks for each playlist

data structure: <br>
tracks_df:      contains each track and all infos on track <br>
playlist_df:    contains all tracks of a playlist


In [6]:
playlist_df = pd.DataFrame(my_playlists.items(), columns=['name', 'id'])
playlist_df.to_csv('data/playlists_df.csv', index=False, header=True)


clean playlist_df:
- only include monthly playlists
or
- only include playlists created by me - chose that for now
- and remove type=episode (has no audio features) + add extra check for audio features -> on track level

In [11]:
for playlist_id in list(my_playlists.values()):
    playlist = sp.playlist(playlist_id)
    if playlist['owner']['id'] != '*lotti_karotti*':
        my_playlists.pop(playlist['name'])

my_playlists_df = pd.DataFrame(my_playlists.items(), columns=['name', 'id'])

In [ ]:
my_playlists_df['track_ids'] = my_playlists_df['id'].apply(lambda x: [track['track']['id'] for track in sp.playlist_tracks(x)['items']])
playlist_df.to_csv('data/my_playlists_df.csv', index=False, header=True)
my_playlists_df.head()

,name,id,track_ids
0,Dezember 2024,6ZYkzdEakzOe8quw3tiPod,"[45G5DwgpBq8Sr6o1dFuO2I, 3hlGuz3loYoLfI3bpwieW..."
1,November 2024,7kZRjiX6iS2wUcr8emNKkb,"[2EbO8k2jAHVRMVvUB77nNt, 102xUWbf0uuklX8qThPeH..."
2,Oktober 2024,1owWVgCwLY4CZ82axZHKEG,"[3iLSSpIswSBF0RiR3KUB9d, 3logNIIMhLuY9A70jyChj..."
3,September 2024,5iDe72U4QklDRv9aJc24sU,"[62ZeAQ8DHdCXJqPQVOTsKu, 45c4L5n0wh6KRv0SOf879..."
4,My recommendation playlist,4r0R7EfW8E2xYyYinzbjYY,"[3qkKAfqOn7laGPeVicY1bX, 1iBWjQ9af9NnUd5xnQDB3..."


In [19]:
tracks = [] # list of track ids (unique) from all playlists
for playlist_name, playlist_id in my_playlists.items():
    playlist = sp.playlist_tracks(playlist_id)
    for track in playlist['items']:
        track_id = track['track']['id']
        if track_id not in tracks and track['track']['type'] == 'track':
            tracks.append(track_id)

## 1.4 get audio features for each track

In [29]:
tracks[5]


'4MIGelOCVbHz4F0NBN8D1c'

In [ ]:
# spotify does no longer allow these calls - need to find a workaround

In [ ]:
# tracks_df = pd.DataFrame(tracks, columns=['track_id'])
# tracks_df.to_csv('data/tracks_df.csv', index=False, header=True)

# audio_features = []
# for i in range(math.ceil(len(tracks)/100)):
#     features = sp.audio_features(tracks[i*100:(i+1)*100])
#     for feature in features:
#         if feature is not None:
#             audio_features.append(feature)


# tracks_df = pd.concat([tracks_df, pd.DataFrame(audio_features)], axis=1)
# tracks_df.to_csv('data/tracks_df.csv', index=False, header=True)


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=45G5DwgpBq8Sr6o1dFuO2I,3hlGuz3loYoLfI3bpwieWq,4a3NAwJX2YTkqLqijLz7I2,6leGtOqboOlMNROa5CP3bl,5w2urkV0oCHiBCo5W2fYmR,4MIGelOCVbHz4F0NBN8D1c,52zlr5j372qsgHhmvrni4w,2eKZvfQGKWrpASz67UCvTo,4Lfp5tapFcY87K1z5s8d4E,6sPpUfwcYBgW3kTXStNsVM,5X7CdXeQ0uyXgy0mYnVpbn,49AG4MU3Ij6a8NWEVLMuRu,0jHnEevYYvE7B2YJoBwz1S,2EbO8k2jAHVRMVvUB77nNt,102xUWbf0uuklX8qThPeHK,485CBwkSIWfEj501d823x6,1rfutWDGqEr4En3aez0J7k,3i86MS9vF1VCf0h2IEpKWY,0L5XzaKc4J86e4iI9BvtIu,1KUwL3m7t3SikjmdbjKk0i,6bCE0p7QX3Tx4rZGDyaJ1U,2GvWK1w9SpqNda6q2l1k31,0yraJk6n1bW91HiOeZZp8k,726SEjbF3ToEbYbn43gnxl,7a4O4FPGjTL4084bbvFeJF,5v66VjrhFwSlPqadCrubyb,465WMnAsGX9twpLm90QgWO,1AlPTZ69vdIwEFKaK9KKY3,5gM6XHLOdzgWq17yd0bE2o,3iLSSpIswSBF0RiR3KUB9d,3logNIIMhLuY9A70jyChjD,4dWoA0dCj9x5YtB3EQQRDi,2XHskR7UoSK6EbxogqqoPx,0FcVZWtHJ0XoHhbT8fiyAg,3uTsiaNxy6h6EAd7MsCf9D,3qcreqi8RaGqahtJSQDxNI,62ZeAQ8DHdCXJqPQVOTsKu,3b1s3Ywg0laGsNHCpUBhrl,2YfrZ7Vpivn9omcPaLo3gm,00aFEMSyan6KOWMDJLtCE8,7bkUa9kDFGx

SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=45G5DwgpBq8Sr6o1dFuO2I,3hlGuz3loYoLfI3bpwieWq,4a3NAwJX2YTkqLqijLz7I2,6leGtOqboOlMNROa5CP3bl,5w2urkV0oCHiBCo5W2fYmR,4MIGelOCVbHz4F0NBN8D1c,52zlr5j372qsgHhmvrni4w,2eKZvfQGKWrpASz67UCvTo,4Lfp5tapFcY87K1z5s8d4E,6sPpUfwcYBgW3kTXStNsVM,5X7CdXeQ0uyXgy0mYnVpbn,49AG4MU3Ij6a8NWEVLMuRu,0jHnEevYYvE7B2YJoBwz1S,2EbO8k2jAHVRMVvUB77nNt,102xUWbf0uuklX8qThPeHK,485CBwkSIWfEj501d823x6,1rfutWDGqEr4En3aez0J7k,3i86MS9vF1VCf0h2IEpKWY,0L5XzaKc4J86e4iI9BvtIu,1KUwL3m7t3SikjmdbjKk0i,6bCE0p7QX3Tx4rZGDyaJ1U,2GvWK1w9SpqNda6q2l1k31,0yraJk6n1bW91HiOeZZp8k,726SEjbF3ToEbYbn43gnxl,7a4O4FPGjTL4084bbvFeJF,5v66VjrhFwSlPqadCrubyb,465WMnAsGX9twpLm90QgWO,1AlPTZ69vdIwEFKaK9KKY3,5gM6XHLOdzgWq17yd0bE2o,3iLSSpIswSBF0RiR3KUB9d,3logNIIMhLuY9A70jyChjD,4dWoA0dCj9x5YtB3EQQRDi,2XHskR7UoSK6EbxogqqoPx,0FcVZWtHJ0XoHhbT8fiyAg,3uTsiaNxy6h6EAd7MsCf9D,3qcreqi8RaGqahtJSQDxNI,62ZeAQ8DHdCXJqPQVOTsKu,3b1s3Ywg0laGsNHCpUBhrl,2YfrZ7Vpivn9omcPaLo3gm,00aFEMSyan

## 1.5 get additional features for each track

In [32]:
tracks_df = pd.read_csv('data/tracks_df.csv')

In [34]:
# features = []

# for uri in tracks_df['uri']:
#     track = sp.track(uri)
#     name = track['name']
#     artist = track['artists'][0]['name']
#     artist_id = track['artists'][0]['id']
#     artist_info = sp.artist(track['artists'][0]['id'])
#     popularity = track['popularity']
#     album = track['album']['name']
#     album_id = track['album']['id']


## medium article code snippets


get info from track id

In [ ]:
id = 'spotify:track:5T0BeNEv0U7xuiEwlCFBCc'
track = sp.track(id)
print(track['name'])

### extracting tracks from a playlist

In [ ]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

extract the name of each track, the name of the album that it belongs to, and the popularity of the track

In [ ]:
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    
    #Album
    album = track["track"]["album"]["name"]
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]

### extracting audio features from a track

In [ ]:
sp.audio_features(track_uri)[0]